### K-최근접 이웃 (K-Nearest Neighbors, KNN)

- **분류**
    - 추론할 feature들과 가까운 feature들로 구성된 data point K 개의 y중 다수의 class로 추론한다.
- **회귀**
    - 추론할 feature들과 가까운 feature들로 구성된 data point K 개의 y값의 평균값으로 추론한다.

| 알고리즘 | 유형 | 주요 특징 | 주요 하이퍼파라미터 |
|-----------|--------|------------|-------------------------|
| **KNN (K-Nearest Neighbors)** | 분류/회귀 | 거리 기반, 학습은 빠르고 예측은 느림, 이상치에 민감 | `n_neighbors= K`, `weights` (`uniform`, `distance`), `metric` (`euclidean`, `manhattan` 등) |


### 머신러닝 모델 개발 및 평가 단계 (전체 흐름도) <br>
<center>
[1. 데이터 준비 및 X,y 생성]<br>
        ↓<br>
[2. 학습/검증용 데이터 분할 (Train/Test Split)]<br>
        ↓<br>
[3. 전처리 및 모델 pipeline 만들기]<br>
[4. 하이퍼파라미터 탐색 및 모델 튜닝 (GridSearchCV)]<br>
        ↓<br>
[5. 모델 학습 (Fit)]<br>
        ↓<br>
[6. 학습 결과 평가 (Best Params, Score)]<br>
        ↓<br>
[7. 테스트셋 최종 평가 (Best Estimator Predict)]
</center>

#### 1. 데이터 준비 및 X,y 생성

In [18]:
import pandas as pd

df = pd.read_csv("data/garments_worker_productivity.csv")

X = df.drop(columns=['actual_productivity', 'date']).values
y = df['actual_productivity'].values

#### 2. 학습/검증용 데이터 분할 (Train/Test Split)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

#### 3. 전처리 및 모델 pipeline 만들기 <br> 4. 하이퍼파라미터 탐색 및 모델 튜닝 (GridSearchCV)

In [20]:
categorical_columns = ['quarter','department','day']
numeric_columns = ['team','targeted_productivity','smv','wip','over_time','incentive','idle_time','idle_men','no_of_style_change','no_of_workers']
target = "actual_productivity"

categorical_columns_index = [1, 2, 3]
numeric_columns_index = [4, 5, 6,7,8,9,10,11,12,13]
# 전처리 파이프라인
# 범주형: 결측치 처리- 최빈값 & OneHot Encoding
# 수치형: 결측치 처리 - 중앙값 & Standard Scaling

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer   # 결측치값 대체.
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

### 범주형 컬럼들에 적용할 전처리 프로세스
cate_pipeline = Pipeline([                        # 범주형 컬럼을 위한 파이프라인 
    ("imputer", SimpleImputer(strategy="most_frequent")), 
    ("ohe", OneHotEncoder(handle_unknown='ignore'))   # handle_unknown='ignore' - 학습할 때 없었던 class는 0으로 처리.  # .default는 exception 발생
])

num_pipeline = Pipeline([                     # 수치형 컬럼을 위한 파이프라인 
    ("imputer", SimpleImputer(strategy="median")), # 1. 결측치 처리
    ("scaler", StandardScaler())   # 2. Feature Scaling
])

# ColumnTransformer로 두개의 전처리 프로세스를 합친다.
preprocessor = ColumnTransformer([
    ("category", cate_pipeline, categorical_columns_index),  #(설정이름, 전처리기, 적용한 컬럼 index)
    ("number", num_pipeline, numeric_columns_index)
])

In [21]:
preprocessor

ColumnTransformer(transformers=[('category',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 [1, 2, 3]),
                                ('number',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 [4, 5, 6, 7, 8, 9, 10, 11, 12, 13])])

In [22]:
### GridSearchCV로 최적 K값, p값 찾기
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("knn", KNeighborsRegressor())     #회귀모델
])
params = {
    "knn__n_neighbors":range(3, 10), 
    "knn__p":[1, 2]
}
gs = GridSearchCV(pipeline, params, scoring="neg_mean_squared_error", cv=4, n_jobs=-1)

#### 5. 모델 학습 (Fit)

In [ ]:
gs.fit(X_train, y_train)

#### 6. 학습 결과 평가 (Best Params, Score)

In [15]:
gs.best_score_
gs.best_params_

{'knn__n_neighbors': 9, 'knn__p': 2}

#### 7. 테스트셋 최종 평가 (Best Estimator Predict)

gs.best_estimator_

In [16]:
from metrics import print_regression_metrcis

best_model = gs.best_estimator_

pred = best_model.predict(X_test)
print_regression_metrcis(y_test, pred, "최종평가")

최종평가
MSE: 0.02057989628402692
RMSE: 0.1434569492357443
R Squared: 0.29254367516292035
